<a href="https://colab.research.google.com/github/ZyadAshraf7/Data-Science-Project/blob/main/ssipynb.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import files
uploaded = files.upload()

for fn in uploaded.keys():
  print('User uploaded file "{name}" with length {length} bytes'.format(
      name=fn, length=len(uploaded[fn])))
  
# Then move kaggle.json into the folder where the API expects to find it.
!mkdir -p ~/.kaggle/ && mv kaggle.json ~/.kaggle/ && chmod 600 ~/.kaggle/kaggle.json

Saving kaggle.json to kaggle.json
User uploaded file "kaggle.json" with length 67 bytes


In [ ]:
!kaggle datasets download -d andrewmvd/leukemia-classification

In [ ]:
from zipfile import ZipFile
file_name = "/content/leukemia-classification.zip"
with ZipFile(file_name,'r') as zip:
  zip.extractall()
  print('Done unzip files...')

In [ ]:
#imports

import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import os
import PIL
from PIL import Image
import tensorflow as tf
from tensorflow.keras.layers import Input, Lambda, Conv2D
from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.python.keras.layers import Dense, Flatten
from tensorflow.keras.models import Sequential
from tensorflow.keras.models import Model
from tensorflow.keras.optimizers import Adam
import random
import cv2
import matplotlib.pyplot as plt
from matplotlib.pyplot import imshow
from timeit import default_timer as timer
from tensorflow.keras.models import Sequential
from keras.utils.np_utils import to_categorical
from tensorflow.keras.preprocessing import image
from tensorflow.keras import models, layers, optimizers
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.layers import Input, Dense, Activation, Flatten, Conv2D, MaxPool2D ,Dropout



In [ ]:
#reading data from its files
all_0 = "/content/C-NMC_Leukemia/training_data/fold_0/all"
all_1 = "/content/C-NMC_Leukemia/training_data/fold_1/all"
all_2 = "/content/C-NMC_Leukemia/training_data/fold_2/all"
hem_0 = "/content/C-NMC_Leukemia/training_data/fold_0/hem"
hem_1 = "/content/C-NMC_Leukemia/training_data/fold_1/hem"
hem_2 = "/content/C-NMC_Leukemia/training_data/fold_2/hem"
path_val ='/content/C-NMC_Leukemia/validation_data/C-NMC_test_prelim_phase_data'
val_labels = pd.read_csv('/content/C-NMC_Leukemia/validation_data/C-NMC_test_prelim_phase_data_labels.csv')


In [ ]:
def get_path_image(folder):
    image_paths = []
    image_fnames = os.listdir(folder) 
    for img_id in range(len(image_fnames)):
        img = os.path.join(folder,image_fnames[img_id])
        image_paths.append(img)
    
    return image_paths
    
cancer_lst = []

for i in [all_0,all_1,all_2]:
    paths = get_path_image(i)
    cancer_lst.extend(paths)
print('No. of cancer images:', len(cancer_lst))

normal_lst = []
for i in [hem_0,hem_1,hem_2]:
    paths = get_path_image(i)
    normal_lst.extend(paths)
print('No. of normal images:', len(normal_lst))


In [ ]:
cancer_dict = {"x_col":cancer_lst, "y_col":[np.nan for x in range(len(cancer_lst))]}
cancer_dict["y_col"] = "ALL"
#print(cancer_dict)

normal_dict = {"x_col":normal_lst, "y_col":[np.nan for x in range(len(normal_lst))]}
normal_dict["y_col"] = "HEM"


cancer_df = pd.DataFrame(cancer_dict)
normal_df = pd.DataFrame(normal_dict)

# select_normal = np.random.choice(normal_lst, 3, replace = False)
# select_all = np.random.choice(cancer_lst, 3, replace = False)

print(cancer_df)

In [ ]:
#Reading the training data
train_df = cancer_df.append(normal_df, ignore_index=True)
#shuffling
train_df = train_df.sample(frac = 1)
#print(train_df)
#Reading the validation data -> not labeled
validation_list = get_path_image(path_val)

#Convert the list to a dictionary. The labels are stored in the val_labels
#we create a dictionary with x as the file name and y as the labels
#The labels are having 0's and 1's.
#0 means normal and 1 means cancer - ALL

validation_dict = {"x_col":validation_list ,"y_col":val_labels["labels"]}
#print(validation_dict)
validation_df = pd.DataFrame(validation_dict)
#print(validation_df)
#1 -> ALL , 0 -> HEM
validation_df["y_col"].replace(to_replace = [1,0], value = ["ALL","HEM"], inplace = True)


In [ ]:
test_data = '/content/C-NMC_Leukemia/testing_data/C-NMC_test_final_phase_data'
test_list = get_path_image(test_data)

test_dict = {"x_col":test_list}
test_df = pd.DataFrame(test_dict)

In [ ]:
#pre
train_datagen = ImageDataGenerator(rescale=1./255)
train_generator = train_datagen.flow_from_dataframe(
                  train_df,
                  x_col = "x_col",
                  y_col = "y_col",
                  target_size = (224, 224), 
                  batch_size = 32,
                  color_mode = "rgb",
                  shuffle = True,
                  class_mode = "binary"
)

In [ ]:
val_datagen = ImageDataGenerator(rescale=1./255)
validation_generator = val_datagen.flow_from_dataframe(
                  validation_df,
                  x_col = "x_col",
                  y_col = "y_col",
                  target_size = (224, 224),                  
                  batch_size = 32,
                  color_mode = "rgb",
                  shuffle = True,
                  class_mode = "binary")

In [ ]:
test_datagen = ImageDataGenerator(rescale=1./255 )
test_generator = test_datagen.flow_from_dataframe(
                  test_df,
                  x_col = "x_col",
                  target_size = (244, 224),
                  color_mode = "rgb",
                  class_mode = None,
                  shuffle = False
)

In [ ]:
#free memory from DFs
del test_df
del train_df
del validation_df

In [ ]:
model = Sequential()
# input_shape = (224, 244, 3)
# input_img = Input(shape= input_shape, name = 'img_input')
pretrained_model = tf.keras.applications.ResNet50(
    include_top=False,
    input_shape=(224,224,3),
    pooling='max',
    classes=2,
    weights='imagenet'
)







In [ ]:
#freezing
for layer in pretrained_model.layers:
    layer.trainable = False

In [ ]:
#Fully Connected layer

model.add(pretrained_model)
model.add(Flatten())
model.add(Dense(64,activation='relu',name ='fully_layer1'))
model.add(Dense(32,activation='relu',name ='fully_layer2'))
model.add(Dense(16,activation='relu',name ='fully_layer3'))
model.add(Dense(5,activation='relu' ,name ='fully_layer4'))
model.add(Dropout(0.5))

model.add(Dense(1,activation='sigmoid'))

# this layer was missing
# model = tf.keras.Model(inputs=input_img, outputs= model)


In [ ]:
#summary 
model.summary()

In [ ]:
model.compile(optimizer='adam',loss='binary_crossentropy',metrics=['accuracy'])

In [ ]:
# start = timer()
history = model.fit_generator(train_generator,validation_data=validation_generator,epochs=40)
# end = timer()

# time = end - start
# print ( print('Total Time Elapsed: ', int(time//60), ' minutes ', (round(time%60)), ' seconds'))

In [ ]:
def load_image(img_path, show=False):

    img = image.load_img(img_path, target_size=(224, 224))
    img_tensor = image.img_to_array(img)                    # (height, width, channels)
    img_tensor = np.expand_dims(img_tensor, axis=0)         # (1, height, width, channels), add a dimension because the model expects this shape: (batch_size, height, width, channels)
    img_tensor /= 255.                                      # imshow expects values in the range [0, 1]


    return img_tensor


new_img = load_image('/content/C-NMC_Leukemia/testing_data/C-NMC_test_final_phase_data/1077.bmp')
#print(new_img)
y_predict = model.predict(new_img)

if(y_predict>=0.5):
    y_predict=1
    print("this patient has cancer")
else:
    y_predict=0 
    print("this patinet is normal")    

print(y_predict)